In [2]:
!pip install pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import struct

# Start Spark session
spark = SparkSession.builder.appName("NYC_Taxi_Analysis").getOrCreate()


In [5]:
from google.colab import files
uploaded = files.upload()


Saving yellow_tripdata_2023-01.parquet to yellow_tripdata_2023-01.parquet


In [6]:


df = spark.read.parquet("yellow_tripdata_2023-01.parquet")
df.show(5)
df.printSchema()


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2023-01-01 00:32:10|  2023-01-01 00:40:36|            1.0|         0.97|       1.0|                 N|         161|         141|           2|        9.3|  1.0|    0.5|       0.

In [7]:
from pyspark.sql.functions import struct

df_nested = df.withColumn("locations", struct("PULocationID", "DOLocationID"))
df_nested.select("locations").show(truncate=False)


+----------+
|locations |
+----------+
|{161, 141}|
|{43, 237} |
|{48, 238} |
|{138, 7}  |
|{107, 79} |
|{161, 137}|
|{239, 143}|
|{142, 200}|
|{164, 236}|
|{141, 107}|
|{234, 68} |
|{79, 264} |
|{164, 143}|
|{138, 33} |
|{33, 61}  |
|{79, 186} |
|{90, 48}  |
|{113, 255}|
|{237, 239}|
|{143, 229}|
+----------+
only showing top 20 rows



In [11]:
from pyspark.sql.functions import col

df_flattened = df_nested.withColumn("pickup_location", col("locations.PULocationID")) \
                        .withColumn("dropoff_location", col("locations.DOLocationID")) \
                        .drop("locations")

df_flattened.select("pickup_location", "dropoff_location").show(5)


+---------------+----------------+
|pickup_location|dropoff_location|
+---------------+----------------+
|            161|             141|
|             43|             237|
|             48|             238|
|            138|               7|
|            107|              79|
+---------------+----------------+
only showing top 5 rows



In [12]:
df_flattened.write.mode("overwrite").parquet("/content/flattened_output.parquet")


In [13]:
df_flattened.show(5)


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------------+----------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|pickup_location|dropoff_location|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------------+----------------+
|       2| 2023-01-01 00:32:10|  2023-01-01 00:40:36|            1.0|         0.97|     